In [14]:
import random

# 菜的類型
vegetables = ["青菜", "蘋果", "西蘭花", "胡蘿蔔", "茄子"]

# 肉的類型
meats = ["雞肉", "豬肉", "牛肉", "羊肉", "魚肉"]

# 飯的類型
rices = ["白米", "糙米", "炸飯"]

# 生成隨機的訓練數據
def generate_random_sample():
    num_vegetables = random.randint(3, 3)
    num_meats = random.randint(1, 1)
    num_rices = random.randint(1, 1)

    # 隨機選擇菜、肉和飯的組合
    selected_vegetables = random.sample(vegetables, num_vegetables)
    selected_meats = random.sample(meats, num_meats)
    selected_rices = random.sample(rices, num_rices)

    # 計算每個食材的面積
    areas = [(item, random.randint(15, 30)) for sublist in [selected_vegetables, selected_meats, selected_rices] for item in sublist]

    # 隨機生成實際價格
    actual_price = random.randint(80, 80)

    return areas, actual_price

# 生成多個訓練樣本
num_samples = 10
random.seed(0)
training_data = [generate_random_sample() for _ in range(num_samples)]

training_data

[([('西蘭花', 26), ('胡蘿蔔', 21), ('蘋果', 19), ('牛肉', 24), ('糙米', 19)], 80),
 ([('青菜', 18), ('茄子', 26), ('西蘭花', 28), ('牛肉', 25), ('糙米', 21)], 80),
 ([('茄子', 15), ('青菜', 30), ('胡蘿蔔', 25), ('羊肉', 22), ('炸飯', 25)], 80),
 ([('蘋果', 30), ('胡蘿蔔', 18), ('青菜', 24), ('雞肉', 24), ('糙米', 18)], 80),
 ([('青菜', 24), ('胡蘿蔔', 20), ('蘋果', 21), ('魚肉', 20), ('白米', 16)], 80),
 ([('蘋果', 27), ('茄子', 23), ('青菜', 22), ('雞肉', 21), ('炸飯', 28)], 80),
 ([('青菜', 25), ('西蘭花', 21), ('胡蘿蔔', 22), ('雞肉', 15), ('糙米', 23)], 80),
 ([('西蘭花', 22), ('胡蘿蔔', 16), ('青菜', 17), ('雞肉', 15), ('白米', 18)], 80),
 ([('西蘭花', 21), ('青菜', 20), ('胡蘿蔔', 18), ('魚肉', 30), ('白米', 21)], 80),
 ([('西蘭花', 24), ('青菜', 26), ('胡蘿蔔', 28), ('雞肉', 20), ('炸飯', 16)], 80)]

In [15]:
import pandas as pd

df = pd.DataFrame(columns=vegetables + meats + rices + ["實際價格"])

# 將訓練數據轉換為 DataFrame
for areas, actual_price in training_data:
    row = dict(areas + [("實際價格", actual_price)])
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

# nan to 0
df = df.fillna(0)
df

,青菜,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯,實際價格
0,0,19,26,21,0,0,0,24,0,0,0,19,0,80
1,18,0,28,0,26,0,0,25,0,0,0,21,0,80
2,30,0,0,25,15,0,0,0,22,0,0,0,25,80
3,24,30,0,18,0,24,0,0,0,0,0,18,0,80
4,24,21,0,20,0,0,0,0,0,20,16,0,0,80
5,22,27,0,0,23,21,0,0,0,0,0,0,28,80
6,25,0,21,22,0,15,0,0,0,0,0,23,0,80
7,17,0,22,16,0,15,0,0,0,0,18,0,0,80
8,20,0,21,18,0,0,0,0,0,30,21,0,0,80
9,26,0,24,28,0,20,0,0,0,0,0,0,16,80


In [16]:
# apply linear regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# 生成測試數據
test_data = [generate_random_sample() for _ in range(5)]

# 將測試數據轉換為 DataFrame
df_test = pd.DataFrame(columns=vegetables + meats + rices + ["實際價格"])
for areas, actual_price in test_data:
    row = dict(areas + [("實際價格", actual_price)])
    df_test = pd.concat([df_test, pd.DataFrame([row])], ignore_index=True)
df_test = df_test.fillna(0)

# Polynomial Regression
for col in df.columns:
    if col == "實際價格":
        continue
    new_df = df.copy()
    new_df[col] = new_df[col] ** 2
    new_df.rename(columns={col: col + "^2"}, inplace=True)

    model = LinearRegression(fit_intercept=False)
    model.fit(new_df.drop("實際價格", axis=1), new_df["實際價格"])
    # 預測測試數據的價格
    new_test_df = df_test.copy()
    new_test_df[col] = new_test_df[col] ** 2
    new_test_df.rename(columns={col: col + "^2"}, inplace=True)
    new_test_df["預測價格"] = model.predict(new_test_df.drop("實際價格", axis=1))
    display(new_test_df)

,青菜^2,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯,實際價格,預測價格
0,0,20,21,0,16,0,0,0,20,0,0,0,20,80,73.408517
1,676,24,0,0,27,0,0,0,23,0,0,0,19,80,86.299484
2,676,0,24,0,26,0,19,0,0,0,0,24,0,80,89.352115
3,0,22,29,0,22,0,0,27,0,0,28,0,0,80,125.649609
4,900,20,0,29,0,0,0,15,0,0,0,0,16,80,83.369386


,青菜,蘋果^2,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯,實際價格,預測價格
0,0,400,21,0,16,0,0,0,20,0,0,0,20,80,55.641141
1,26,576,0,0,27,0,0,0,23,0,0,0,19,80,71.366744
2,26,0,24,0,26,0,19,0,0,0,0,24,0,80,86.129600
3,0,484,29,0,22,0,0,27,0,0,28,0,0,80,116.411002
4,30,400,0,29,0,0,0,15,0,0,0,0,16,80,90.210997


,青菜,蘋果,西蘭花^2,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯,實際價格,預測價格
0,0,20,441,0,16,0,0,0,20,0,0,0,20,80,74.831219
1,26,24,0,0,27,0,0,0,23,0,0,0,19,80,85.003584
2,26,0,576,0,26,0,19,0,0,0,0,24,0,80,89.511217
3,0,22,841,0,22,0,0,27,0,0,28,0,0,80,118.654502
4,30,20,0,29,0,0,0,15,0,0,0,0,16,80,74.954827


,青菜,蘋果,西蘭花,胡蘿蔔^2,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯,實際價格,預測價格
0,0,20,21,0,16,0,0,0,20,0,0,0,20,80,77.724788
1,26,24,0,0,27,0,0,0,23,0,0,0,19,80,88.362679
2,26,0,24,0,26,0,19,0,0,0,0,24,0,80,86.877889
3,0,22,29,0,22,0,0,27,0,0,28,0,0,80,122.767465
4,30,20,0,841,0,0,0,15,0,0,0,0,16,80,81.701451


,青菜,蘋果,西蘭花,胡蘿蔔,茄子^2,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯,實際價格,預測價格
0,0,20,21,0,256,0,0,0,20,0,0,0,20,80,70.691716
1,26,24,0,0,729,0,0,0,23,0,0,0,19,80,90.689231
2,26,0,24,0,676,0,19,0,0,0,0,24,0,80,88.336865
3,0,22,29,0,484,0,0,27,0,0,28,0,0,80,113.426179
4,30,20,0,29,0,0,0,15,0,0,0,0,16,80,78.806704


...

,青菜,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉^2,魚肉,白米,糙米,炸飯,實際價格,預測價格
0,0,20,21,0,16,0,0,0,400,0,0,0,20,80,71.710453
1,26,24,0,0,27,0,0,0,529,0,0,0,19,80,85.125317
2,26,0,24,0,26,0,19,0,0,0,0,24,0,80,85.723953
3,0,22,29,0,22,0,0,27,0,0,28,0,0,80,114.921496
4,30,20,0,29,0,0,0,15,0,0,0,0,16,80,81.954896


,青菜,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉^2,白米,糙米,炸飯,實際價格,預測價格
0,0,20,21,0,16,0,0,0,20,0,0,0,20,80,73.616297
1,26,24,0,0,27,0,0,0,23,0,0,0,19,80,84.740651
2,26,0,24,0,26,0,19,0,0,0,0,24,0,80,84.356680
3,0,22,29,0,22,0,0,27,0,0,28,0,0,80,114.629296
4,30,20,0,29,0,0,0,15,0,0,0,0,16,80,79.033367


,青菜,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米^2,糙米,炸飯,實際價格,預測價格
0,0,20,21,0,16,0,0,0,20,0,0,0,20,80,53.454161
1,26,24,0,0,27,0,0,0,23,0,0,0,19,80,75.941954
2,26,0,24,0,26,0,19,0,0,0,0,24,0,80,85.841223
3,0,22,29,0,22,0,0,27,0,0,784,0,0,80,146.057736
4,30,20,0,29,0,0,0,15,0,0,0,0,16,80,99.079807


,青菜,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米^2,炸飯,實際價格,預測價格
0,0,20,21,0,16,0,0,0,20,0,0,0,20,80,68.514565
1,26,24,0,0,27,0,0,0,23,0,0,0,19,80,82.449557
2,26,0,24,0,26,0,19,0,0,0,0,576,0,80,80.290658
3,0,22,29,0,22,0,0,27,0,0,28,0,0,80,116.424125
4,30,20,0,29,0,0,0,15,0,0,0,0,16,80,83.037428


,青菜,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯^2,實際價格,預測價格
0,0,20,21,0,16,0,0,0,20,0,0,0,400,80,56.559583
1,26,24,0,0,27,0,0,0,23,0,0,0,361,80,72.974712
2,26,0,24,0,26,0,19,0,0,0,0,24,0,80,76.135859
3,0,22,29,0,22,0,0,27,0,0,28,0,0,80,109.823644
4,30,20,0,29,0,0,0,15,0,0,0,0,256,80,86.243705


In [17]:
# apply linear regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

model = LinearRegression(fit_intercept=False) # otherwise, intercept is 80...
model.fit(df.drop("實際價格", axis=1), df["實際價格"])
model.coef_

array([ 8.48404154e-01,  1.00811188e+00,  9.97234991e-01,  8.73602960e-01,
        7.11786167e-01, -7.41292547e-02, -1.11022302e-16,  1.10258825e-02,
        2.18060458e-01, -3.17861024e-01,  1.70956950e+00,  8.58288481e-01,
        6.89347153e-01])

In [18]:
# 生成測試數據
test_data = [generate_random_sample() for _ in range(5)]

# 將測試數據轉換為 DataFrame
df_test = pd.DataFrame(columns=vegetables + meats + rices + ["實際價格"])
for areas, actual_price in test_data:
    row = dict(areas + [("實際價格", actual_price)])
    df_test = pd.concat([df_test, pd.DataFrame([row])], ignore_index=True)
df_test = df_test.fillna(0)

df_test

,青菜,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯,實際價格
0,17,0,0,19,15,0,0,0,0,27,0,0,28,80
1,21,0,0,23,24,20,0,0,0,0,0,0,18,80
2,19,0,26,18,0,0,0,23,0,0,15,0,0,80
3,0,28,29,0,30,0,0,0,0,26,20,0,0,80
4,0,26,0,25,17,0,0,16,0,0,0,16,0,80


In [19]:
# 預測測試數據的價格
df_test["預測價格"] = model.predict(df_test.drop("實際價格", axis=1))
df_test

,青菜,蘋果,西蘭花,胡蘿蔔,茄子,雞肉,豬肉,牛肉,羊肉,魚肉,白米,糙米,炸飯,實際價格,預測價格
0,17,0,0,19,15,0,0,0,0,27,0,0,28,80,52.417592
1,21,0,0,23,24,20,0,0,0,0,0,0,18,80,65.917887
2,19,0,26,18,0,0,0,23,0,0,15,0,0,80,83.669780
3,0,28,29,0,30,0,0,0,0,26,20,0,0,80,104.427536
4,0,26,0,25,17,0,0,16,0,0,0,16,0,80,74.060378
